<a href="https://colab.research.google.com/github/solanki1993/Covid19-India-Analysis-and-forecasting-using-FbProphet/blob/main/Advanced_covid19_forecasting_using_FbProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
Covid_Testing = pd.read_csv('/content/StatewiseTestingDetails.csv')
Covid_Testing.head()

,Date,State,TotalSamples,Negative,Positive
0,2020-04-17,Andaman and Nicobar Islands,1403.0,1210,12.0
1,2020-04-24,Andaman and Nicobar Islands,2679.0,NaN,27.0
2,2020-04-27,Andaman and Nicobar Islands,2848.0,NaN,33.0
3,2020-05-01,Andaman and Nicobar Islands,3754.0,NaN,33.0
4,2020-05-16,Andaman and Nicobar Islands,6677.0,NaN,33.0


In [3]:
Covid_Testing['State'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'],
      dtype=object)

In [4]:
print(Covid_Testing['Date'].min())
print(Covid_Testing['Date'].max())

2020-04-01
2021-08-10


In [5]:
Covid_Summary = pd.read_csv('/content/covid_19_india.csv')
Covid_Summary.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [7]:
Covid_Summary.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [8]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18110 entries, 0 to 18109
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       18110 non-null  int64 
 1   Date                      18110 non-null  object
 2   Time                      18110 non-null  object
 3   State/UnionTerritory      18110 non-null  object
 4   ConfirmedIndianNational   18110 non-null  object
 5   ConfirmedForeignNational  18110 non-null  object
 6   Cured                     18110 non-null  int64 
 7   Deaths                    18110 non-null  int64 
 8   Confirmed                 18110 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.2+ MB


In [9]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,6:00 PM,Kerala,0,0,1
1,2020-01-31,6:00 PM,Kerala,0,0,1
2,2020-02-01,6:00 PM,Kerala,0,0,2
3,2020-02-02,6:00 PM,Kerala,0,0,3
4,2020-02-03,6:00 PM,Kerala,0,0,3


In [10]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.tail()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
18105,2021-08-11,8:00 AM,Telangana,638410,3831,650353,8112
18106,2021-08-11,8:00 AM,Tripura,77811,773,80660,2076
18107,2021-08-11,8:00 AM,Uttarakhand,334650,7368,342462,444
18108,2021-08-11,8:00 AM,Uttar Pradesh,1685492,22775,1708812,545
18109,2021-08-11,8:00 AM,West Bengal,1506532,18252,1534999,10215


In [11]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'])

In [12]:
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
0,2020-01-30,6:00 PM,Kerala,0,0,1,1
1,2020-01-31,6:00 PM,Kerala,0,0,1,1
2,2020-02-01,6:00 PM,Kerala,0,0,2,2
3,2020-02-02,6:00 PM,Kerala,0,0,3,3
4,2020-02-03,6:00 PM,Kerala,0,0,3,3


In [13]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18110 entries, 0 to 18109
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  18110 non-null  datetime64[ns]
 1   Time                  18110 non-null  object        
 2   State/UnionTerritory  18110 non-null  object        
 3   Cured                 18110 non-null  int64         
 4   Deaths                18110 non-null  int64         
 5   Confirmed             18110 non-null  int64         
 6   Active                18110 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 990.5+ KB


In [14]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [15]:
india_cases.head()

,Cured,Deaths,Confirmed,Active
State/UnionTerritory,,,,
Andaman and Nicobar Islands,7412,129,7548,7
Andhra Pradesh,1952736,13564,1985182,18882
Arunachal Pradesh,47821,248,50605,2536
Assam,559684,5420,576149,11045
Bihar,715352,9646,725279,281


In [16]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

,Cured,Deaths,Confirmed,Active
0,31220981,429179,32036511,386351


In [17]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()

In [18]:
Trend.head()

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0


In [19]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [20]:
import plotly.express as px

In [21]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [23]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index,
                     'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Red')

In [24]:
top_10_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
            'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','Green')

In [25]:
top_10_recovered_states = india_cases.sort_values('Cured', ascending=False)[:10]
horizontal_bar_chart(top_10_recovered_states, top_10_recovered_states.Cured, top_10_recovered_states.index,
                    'Top 10 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Cyan')

In [27]:
#vaccination

vaccination = pd.read_csv(r"/content/covid_vaccine_statewise.csv")
vaccination.head()

,Updated On,State,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male (Doses Administered),Female (Doses Administered),Transgender (Doses Administered),Covaxin (Doses Administered),CoviShield (Doses Administered),Sputnik V (Doses Administered),AEFI,18-44 Years (Doses Administered),45-60 Years (Doses Administered),60+ Years (Doses Administered),18-44 Years(Individuals Vaccinated),45-60 Years(Individuals Vaccinated),60+ Years(Individuals Vaccinated),Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Individuals Vaccinated
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,NaN,NaN,NaN,579.0,47697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23757.0,24517.0,2.0,48276.0
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,NaN,NaN,NaN,635.0,57969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27348.0,31252.0,4.0,58604.0
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,NaN,NaN,NaN,1299.0,98150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41361.0,58083.0,5.0,99449.0
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,NaN,NaN,NaN,3017.0,192508.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81901.0,113613.0,11.0,195525.0
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,NaN,NaN,NaN,3946.0,247334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98111.0,153145.0,24.0,251280.0


In [28]:
vaccination.tail()

,Updated On,State,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male (Doses Administered),Female (Doses Administered),Transgender (Doses Administered),Covaxin (Doses Administered),CoviShield (Doses Administered),Sputnik V (Doses Administered),AEFI,18-44 Years (Doses Administered),45-60 Years (Doses Administered),60+ Years (Doses Administered),18-44 Years(Individuals Vaccinated),45-60 Years(Individuals Vaccinated),60+ Years(Individuals Vaccinated),Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Individuals Vaccinated
7840,11/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7841,12/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7842,13/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7843,14/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7844,15/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [35]:
WB = vaccination[vaccination["State"]=="West Bengal"]
fig = px.line(WB, x="Date",y="Total Vaccinatons",title="Vaccination till date in West Bengal")  
fig.update_xaxes(rangeslider_visible=True)

In [37]:
fig = px.line(WB,x="Date",y="CoviShield (Doses Administered)",title="CoviShield Administered in West Bengal")  
fig.update_xaxes(rangeslider_visible=True) 

In [48]:
fig = px.line(WB,x="Date",y=" Covaxin (Doses Administered)",title="Covaxin Administered in West Bengal")  
fig.update_xaxes(rangeslider_visible=True) 

In [49]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode

In [50]:
model = Prophet()

In [51]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

,Date,Confirmed
555,2021-08-07,31895385
556,2021-08-08,31934455
557,2021-08-09,31969954
558,2021-08-10,31998158
559,2021-08-11,32036511


In [52]:
Cured = Trend.loc[:, ['Date', 'Cured']] 
Cured.tail()

,Date,Cured
555,2021-08-07,31055861
556,2021-08-08,31099771
557,2021-08-09,31139457
558,2021-08-10,31180968
559,2021-08-11,31220981


In [53]:
Confirmed.columns = ['ds', 'y']
model.fit(Confirmed)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [54]:
future = model.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future.tail()

,ds
615,2021-10-06
616,2021-10-07
617,2021-10-08
618,2021-10-09
619,2021-10-10


In [55]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_conf = model.predict(future)
forecast_india_conf


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-2.203121e+05,-1.806932e+06,1.496032e+06,-2.203121e+05,-2.203121e+05,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,0.0,0.0,0.0,-1.991879e+05
1,2020-01-31,-2.167013e+05,-1.832278e+06,1.504786e+06,-2.167013e+05,-2.167013e+05,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,0.0,0.0,0.0,-2.006712e+05
2,2020-02-01,-2.130905e+05,-1.776038e+06,1.289056e+06,-2.130905e+05,-2.130905e+05,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,0.0,0.0,0.0,-2.023903e+05
3,2020-02-02,-2.094797e+05,-1.871370e+06,1.493218e+06,-2.094797e+05,-2.094797e+05,4562.834651,4562.834651,4562.834651,4562.834651,4562.834651,4562.834651,0.0,0.0,0.0,-2.049168e+05
4,2020-02-03,-2.058689e+05,-1.854245e+06,1.372779e+06,-2.058689e+05,-2.058689e+05,-4188.545142,-4188.545142,-4188.545142,-4188.545142,-4188.545142,-4188.545142,0.0,0.0,0.0,-2.100574e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2021-10-06,4.547898e+07,4.357170e+07,4.717864e+07,4.462848e+07,4.626385e+07,-28914.547406,-28914.547406,-28914.547406,-28914.547406,-28914.547406,-28914.547406,0.0,0.0,0.0,4.545007e+07
616,2021-10-07,4.564438e+07,4.380193e+07,4.745716e+07,4.476491e+07,4.645674e+07,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,0.0,0.0,0.0,4.566550e+07
617,2021-10-08,4.580977e+07,4.397317e+07,4.759066e+07,4.490269e+07,4.664866e+07,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,0.0,0.0,0.0,4.582580e+07
618,2021-10-09,4.597517e+07,4.414287e+07,4.789827e+07,4.502032e+07,4.683331e+07,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,0.0,0.0,0.0,4.598587e+07


In [56]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [57]:
model1 = Prophet()

In [58]:
# Dataframe must have columns "Date" and "Values" with the dates and values  for prophet prediction
Cured.columns = ['ds', 'y']
model1.fit(Cured)

future_1 = model1.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future_1.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
615,2021-10-06
616,2021-10-07
617,2021-10-08
618,2021-10-09
619,2021-10-10


In [59]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_cured = model1.predict(future_1)
forecast_india_cured

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-1.437263e+05,-1.473181e+06,1.227723e+06,-1.437263e+05,-1.437263e+05,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,0.0,0.0,0.0,-1.230335e+05
1,2020-01-31,-1.413917e+05,-1.536057e+06,1.245631e+06,-1.413917e+05,-1.413917e+05,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,0.0,0.0,0.0,-1.277022e+05
2,2020-02-01,-1.390571e+05,-1.597324e+06,1.268322e+06,-1.390571e+05,-1.390571e+05,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,0.0,0.0,0.0,-1.312793e+05
3,2020-02-02,-1.367226e+05,-1.430010e+06,1.236985e+06,-1.367226e+05,-1.367226e+05,2123.319301,2123.319301,2123.319301,2123.319301,2123.319301,2123.319301,0.0,0.0,0.0,-1.345992e+05
4,2020-02-03,-1.343880e+05,-1.545875e+06,1.300973e+06,-1.343880e+05,-1.343880e+05,-8339.938716,-8339.938716,-8339.938716,-8339.938716,-8339.938716,-8339.938716,0.0,0.0,0.0,-1.427279e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2021-10-06,4.473314e+07,4.303249e+07,4.627754e+07,4.380995e+07,4.555861e+07,-22141.705314,-22141.705314,-22141.705314,-22141.705314,-22141.705314,-22141.705314,0.0,0.0,0.0,4.471100e+07
616,2021-10-07,4.490559e+07,4.311699e+07,4.662875e+07,4.395538e+07,4.575515e+07,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,0.0,0.0,0.0,4.492628e+07
617,2021-10-08,4.507804e+07,4.339847e+07,4.679366e+07,4.408752e+07,4.594660e+07,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,0.0,0.0,0.0,4.509173e+07
618,2021-10-09,4.525049e+07,4.354048e+07,4.699040e+07,4.423703e+07,4.613783e+07,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,0.0,0.0,0.0,4.525827e+07


In [60]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig)